### This notebook introduces the library h2_station_distributor by providing an overview of some of its functionality

First you need to have the library installed, for that simply move to the folder to which you cloned the repo and run `pip install .`

In [ ]:
# Import some classes
from h2_station_distributor.utils.helpers import Accounting, Data, LoadData, Plots, Strategies

In [ ]:
# Now we can start off by loading important GeoDataFrames
regions = LoadData.load_regions()
traffic = LoadData.load_traffic()
depreg = LoadData.load_depreg()
stations = LoadData.load_stations()
logistic_hubs = LoadData.load_logistic_hubs()
water_bodies = LoadData.load_water_bodies()

### Before we can move on, some GeoDataFrames have to be slightly adapted for our purposes, luckily Data holds the necessary methods for that

In [ ]:
# We are only interested in the areas in mainland France and will therefore remove other records
stations = Data.limit_gdfs_to_main_land_france(df=stations, df_regions=regions)

#### The traffic GeoDataFrames will be expected to have some more information than it originally comes with so let's add that next

In [ ]:
# We want to add the traffic of H2 trucks, for this example let's assume that there will be 10'000 H2 truck on the
# road in 2030
n_trucks = 10_000
traffic = Data.add_h2_truck_traffic(traffic, n_trucks=n_trucks, target_year=2030)
# Now we want to add a column providing information on the H2 truck traffic density which will later be used as a
# prior for the station and plant distribution. As density area we are going to use 50 km
traffic = Data.add_buffer_distance(traffic, buffer_size=50000)
# Since we have data on the logistic hubs in France which will be helpful in deciding on the location of stations,
# we add it now
traffic = Data.add_distance_to_nearest_logistic_hub(traffic, logistic_hubs)
# Finally, since H2 production plants need a lot of water, we are going to consider the proximity to a body of
# of water when we distributing the plants, luckily the internet provided us with this data which we integrate now
traffic = Data.add_distance_to_nearest_body_of_water(traffic_hub_distance, water_bodies)

### Now that we have everything to distribute stations and plants let's jump right into it
##### For the purpose of demonstration we created some simpler methods to distribute stations and plants to get a quick overview of what that might look like

In [ ]:
# First we create the station distribution (for example for 200 stations)
df_areas_stations = Strategies.split_into_n_areas(traffic, n_areas=147, buffer_step=5000)
station_distribution = Strategies.create_h2_station_distribution_from_areas(df_areas=df_areas_stations,
                                                                            df_with_hub_distance=traffic)
# Now we can distribute the plants (for this example only big ones)
df_areas_plants = Strategies.split_into_areas_fuel_based(traffic, h2_in_kg=48000)
plant_distribution = Strategies.create_h2_plant_distribution_from_areas(df_areas=df_areas_plants,
                                                                        df_hubs_water=traffic)

### That's already it, so let's visualize our results

You can either create matplotlib figures:

In [ ]:
# Plotting the stations
Plots.plot_chosen_stations(station_distribution, regions)
# Plotting the plants
Plots.plot_chosen_plants(plant_distribution, regions)

Or you can plot plotly figures:

In [ ]:
# Plotting the stations
Plots.plotly_plot_chosen_stations(station_distribution, regions)
# Plotting the plants
Plots.plotly_plot_chosen_plants(plant_distribution, regions)

This notebook is only a short introduction to the library, feel free to explore it in more depth and (if you have
time) run the more complex algorithms which can be found in the "models" folder and in the "RL" class in the "helpers" module